In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\akish.pothuri\\python\\MedicalChatBot'

In [2]:
%pip install pyodbc

   ---------------------------------------- 0.0/68.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.8 kB ? eta -:--:--
   ----------------- ---------------------- 30.7/68.8 kB ? eta -:--:--
   ---------------------------------------- 68.8/68.8 kB 622.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyodbc 

In [19]:
import pyodbc
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=GGKU4DELL1254;Database=projectcodefirst;Trusted_Connection=yes;')

In [25]:
cnxn.cursor()

In [9]:
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM Employees")

In [10]:
row = cursor.fetchall()

In [11]:
row

[(1, 'Madhuaddon,addon', 1),
 (2, 'Aniladdon,addon', 2),
 (3, 'Amiraddon,addon', 3)]

In [ ]:
# pip install google-generativeai
#pip install python-dotenv

In [5]:
import os
from langchain.llms import GooglePalm
from dotenv import load_dotenv 
load_dotenv()


True

In [8]:
palm_key = os.getenv('api_key')

In [11]:
llm = GooglePalm(google_api_key=palm_key, temperature=0.3)

In [13]:
code = llm("Code to linear regression in machine learning algorithms")

In [16]:
code = llm("Fetch all from the Employees table")

In [17]:
print(code)

```sql
SELECT *
FROM Employees;
```


In [28]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine

In [29]:
# check the local ODBC driver and make sure it match with the traget database instance
import pyodbc
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [34]:
#windows authentication connection string.
db = SQLDatabase.from_uri(
    database_uri="mssql+pyodbc://[server]/[database]?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [35]:
sql_toolkit=SQLDatabaseToolkit(db=db,llm=llm)
sql_toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E0727A0E80>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E0727A0E80>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E0727A0E80>),
 QuerySQLCheckerTool(description='Use this tool to 

In [36]:
print(db.get_table_names())

['Customer', 'Order', 'OrderItem', 'Product', 'Supplier']


c:\Users\akish.pothuri\python\LangChain\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `get_table_names` was deprecated in LangChain 0.0.1 and will be removed in 0.2.0. Use get_usable_table_name instead.
  warn_deprecated(


In [37]:
print(db.get_table_info())


CREATE TABLE [Customer] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[FirstName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[LastName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_CUSTOMER] PRIMARY KEY ([Id])
)

/*
3 rows from Customer table:
Id	FirstName	LastName	City	Country	Phone
1	Maria	Anders	Berlin	Germany	030-0074321
2	Ana	Trujillo	México D.F.	Mexico	(5) 555-4729
3	Antonio	Moreno	México D.F.	Mexico	(5) 555-3932
*/


CREATE TABLE [OrderItem] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[OrderId] INTEGER NOT NULL, 
	[ProductId] INTEGER NOT NULL, 
	[UnitPrice] DECIMAL(12, 2) NOT NULL DEFAULT ((0)), 
	[Quantity] INTEGER NOT NULL DEFAULT ((1)), 
	CONSTRAINT [PK_ORDERITEM] PRIMARY KEY ([Id]), 
	CONSTRAINT [FK_ORDERITE_REFERENCE_ORDER] FOREIGN K

In [45]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("what is the total number of customers")



> Entering new SQLDatabaseChain chain...
what is the total number of customers
SQLQuery:SELECT COUNT(*) FROM Customer
SQLResult: [(91,)]
Answer:91
> Finished chain.


In [46]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("what are the names of the customers from germany courntry?")



> Entering new SQLDatabaseChain chain...
what are the names of the customers from germany courntry?
SQLQuery:SELECT FirstName, LastName FROM Customer WHERE Country = 'Germany'
SQLResult: [('Maria', 'Anders'), ('Hanna', 'Moos'), ('Sven', 'Ottlieb'), ('Peter', 'Franken'), ('Philip', 'Cramer'), ('Renate', 'Messner'), ('Alexander', 'Feuer'), ('Henriette', 'Pfalzheim'), ('Horst', 'Kloss'), ('Karin', 'Josephs'), ('Rita', 'Müller')]
Answer:Maria Anders, Hanna Moos, Sven Ottlieb, Peter Franken, Philip Cramer, Renate Messner, Alexander Feuer, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Rita Müller
> Finished chain.


In [48]:
qns1 = db_chain("which company providing more products?")




> Entering new SQLDatabaseChain chain...
which company providing more products?
SQLQuery:SELECT CompanyName FROM Supplier GROUP BY CompanyName ORDER BY COUNT(*) DESC
SQLResult: [('Aux joyeux ecclésiastiques',), ('Bigfoot Breweries',), ("Cooperativa de Quesos 'Las Cabras'",), ('Escargots Nouveaux',), ('Exotic Liquids',), ("Forêts d'érables",), ('Formaggi Fortini s.r.l.',), ('Gai pâturage',), ("G'day, Mate",), ("Grandma Kelly's Homestead",), ('Heli Süßwaren GmbH & Co. KG',), ('Karkki Oy',), ('Leka Trading',), ('Lyngbysild',), ('Ma Maison',), ("Mayumi's",), ('New England Seafood Cannery',), ('New Orleans Cajun Delights',), ('Nord-Ost-Fisch Handelsgesellschaft mbH',), ('Norske Meierier',), ('Pasta Buttini s.r.l.',), ('Pavlova, Ltd.',), ('PB Knäckebröd AB',), ('Plutzer Lebensmittelgroßmärkte AG',), ('Refrescos Americanas LTDA',), ('Specialty Biscuits, Ltd.',), ('Svensk Sjöföda AB',), ('Tokyo Traders',), ('Zaanse Snoepfabriek',)]
Answer:Exotic Liquids
> Finished chain.
